<a target="_blank" href="https://colab.research.google.com/github/Gogonemnem/Safran-Project/blob/main/notebooks/llm_notebook_exp.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip uninstall tensorflow-probability -y
!pip install chromadb accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
WORKING_DIR = "/path/to/your/data"

In [ ]:
from huggingface_hub import login
access_token_write = "hf_yourAccessToken"
login(token = access_token_write)

# IMPORTS

## PACKAGES

In [ ]:
from chromadb.utils.embedding_functions import DefaultEmbeddingFunction
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from sklearn import metrics
import os
import re
import numpy as np
import pandas as pd
import chromadb
import transformers
import torch
import pickle as pkl
import joblib
from itertools import chain

## MODEL

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float32,
    device_map="auto",
)

# FUNCTIONS DEFINITION

## PROMPT EXAMPLES
f"""Consider three examples [EXAMPLE 1], [EXAMPLE 2], [EXAMPLE 3], instances of narratives from NASA's ASRS database labeled as {label}". Your objective is to assess if a [NEW NARRATIVE] can be assigned the same label as these examples. Return 1 if it can, and 0 otherwise."""

f"""Given threes examples [EXAMPLE 1], [EXAMPLE 2], [EXAMPLE 3] which are instances of narratives from NASA's ASRS database labeled as "{label}", your task is to determine whether a [NEW NARRATIVE] can be assigned the same label as these examples. Return 1 if the [NEW NARRATIVE] can be assigned the same label as the examples and 0 otherwise."""

f"""Consider three examples [EXAMPLE 1], [EXAMPLE 2], [EXAMPLE 3], instances of narratives from NASA's ASRS database labeled as {label}". Your objective is to assess if a [NEW NARRATIVE] can be assigned the same label as these examples. As [Answer] return 1 if it can, and 0 otherwise."""  #  Only provide the numerical result (0 or 1) and no additional information.

"""<<SYS>> As an aviation safety specialist, you will be provided with an input, which is the narrative from Aviation Safety reports of the NASA's ASRS dataset. Tell me if the input corresponds to which anomalies among the following categories:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

Only return the corresponding anomalies as a 14-length onehot encoding list where each indice corresponds to the categorie with the same indice. For example, if the input corresponds to <<0. 'Deviation / Discrepancy - Procedural'>>, put 1 at indice 0 of the 14-length onehot encoding list like this : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]. The input can correspond to one or many of the categories, so put 1 at their respective indices and 0 elsewhere. <<SYS>>"""

"""<<SYS>> As an aviation safety specialist, you will be provided with an input, which is the narrative from Aviation Safety reports of the NASA's ASRS dataset. Tell me if the input corresponds to which anomalies among the following categories:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

Only return the corresponding anomalies and nothing else. <<SYS>>"""

"""As an aviation safety specialist, you will be provided with an [NARRATIVE], which is the narrative from Aviation Safety reports of the NASA's ASRS database.
You task is to assess if the [NARRATIVE] contains enough elements to be labelled as {label}. To help you, you will be provided 2 examples: [EXAMPLE1] is labelled as {label} and [EXAMPLE2] as {label2}.
Return [YES] or [NO] the [NARRATIVE] has enough elements to be labelled as {label} and the probability about how much your are confident about that.
Note that there is a total of 14 labels in the hole database."""

""" You are an aviation safety expert. Your job is to find out aviation safety related concerns from a given [NARRATIVE] from Aviation Safety reports of the NASA's ASRS database. Here is the list of categories of concerns :
0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'
Please read the narrative and indicate whether each concern is mentioned. In your response, provide a map with keys corresponding to the list of concern categories and values representing the probability of each concern being mentioned in the narrative. Here is an example of the expected response:
{'Deviation / Discrepancy - Procedural' : 0.85,
'Aircraft Equipment': 0.95,
'Conflict' : 0.50,
'Inflight Event / Encounter' : 0.0,
'ATC Issue' : 0.6,
'Deviation - Altitude' : 0.0,
'Deviation - Track / Heading' : 0.1,
'Ground Event / Encounter' : 0.2,
'Flight Deck / Cabin / Aircraft Event' : 0.0,
'Ground Incursion' : 0.0,
'Airspace Violation': 0.0,
'Deviation - Speed' : 0.0,
'Ground Excursion' : 0.05,
'No Specific Anomaly Occurred' : 0.01}
"""

f"""As an aviation safety specialist, you will be provided with an narrative [NARRATIVE], which is a narrative from Aviation Safety reports of the NASA's ASRS database.
You task is to assess if the narrative [NARRATIVE] can be labeled as <<{label}>>. To help you, you will be provided 2 examples: the example [EXAMPLE1] is labeled as <<{label}>> and the example [EXAMPLE2] is labeled as <<{label2}>>. Based on these examples, just return
the label of the narrative [NARRATIVE].
Note that there is a total of 14 labels in the hole database. Following all the label in the database :
0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'
"""

f"""You are an aviation safety expert. Your task is to determine whether there are any aviation safety concerns present in the provided [NARRATIVE], sourced from the NASA ASRS database, specifically related to <<{label}>>. [EXAMPLE1] and [EXAMPLE2] illustrate narratives containing concerns related to <<{label}>>. Please review [NARRATIVE] and indicate whether it contains any concerns related to <<{label}>>. In your response, provide a map with 'yes' and 'no' as keys, where the values represent their respective probabilities. The sum of these probabilities must equal one. Below is an example of the expected response:
['yes' : 0.85, 'no': 0.15]
"""

## FUNCTIONS

In [ ]:
def labeller(**kwargs):
    """For each Anomaly, return {Anomaly:1} and {Others:1} if the cell is empty.
    """
    for name, item in kwargs.items():
        if isinstance(item, str):
            cell = item
        if 'anomalies' in name.lower() and isinstance(item, list):
            anomalies_list = item
    cell_labels =[]
    cell_notin = []
    for prefix in anomalies_list:
        if any(anomaly.strip().startswith(prefix) for anomaly in cell.split(';')):
            cell_labels.append(prefix)

        else:
            cell_notin.append(prefix)
    metadata = dict(zip(cell_labels + cell_notin, [1]*len(cell_labels) + [0]*len(cell_notin)))
    return metadata


def load_from_df(**kwargs):
    """return the narratives and their Anomalies in two list.
    """
    for name, item in kwargs.items():
        if "docs" in name.lower():
            source = item
        if "metadata" in name.lower():
            meta = item
        if isinstance(item, pd.DataFrame):
            data = item
        if 'anomalies' in name.lower() and isinstance(item, list):
            anomalies_list = item
    documents = data[source].apply(lambda cell: cell.strip()).values.tolist()
    metadata = data[meta].apply(lambda cell: labeller(input=cell, anomalies_list=anomalies_list)).values.tolist()
    return documents, metadata

In [ ]:
def format_zsinference(**kwargs):
  for name, item in kwargs.items():
    if "inference" in name.lower():
      inference = item
    if "threshold" in name.lower():
      t = item
  pattern1 = re.compile(r'\[INST\].*?\[/INST\]', re.DOTALL)
  pattern2 = re.compile(r"'([^']+)'\s*:\s*(\d+(\.\d+)?)", re.DOTALL)
  rm_pattern1 = re.sub(pattern1, '', inference[0]['generated_text'].strip())
  extract_pattern2 = re.findall(pattern2, rm_pattern1)
  preds = [1 if float(prob) > t else 0 for _, prob, _ in extract_pattern2]
  if len(preds) != 14:
    preds = [0] * 14
  preds = np.array(preds)
  return preds, rm_pattern1


def format_fsinference(**kwargs):
  for name, item in kwargs.items():
    if "inference" in name.lower():
      inference = item
    if "threshold" in name.lower():
      t = item
  pattern1 = re.compile(r'\[INST\].*?\[/INST\]', re.DOTALL)
  pattern2 = re.compile(r"'([^']+)'\s*:\s*(\d+(\.\d+)?)", re.DOTALL)
  rm_pattern1 = re.sub(pattern1, '', inference[0]['generated_text'].strip())
  extract_pattern2 = re.findall(pattern2, rm_pattern1)
  preds = [1 if float(prob) > t else 0 for _, prob, _ in extract_pattern2]
  if len(preds) != 2:
    preds = [0] * 2
  preds = np.array(preds)
  return preds, rm_pattern1


def get_fsprompt(**kwargs):
    """Few shot prompt builder"""
    for name, item in kwargs.items():
        if 'example1' in name.lower():
            example1 = item
        if 'example2' in name.lower():
            example2 = item
        if 'narrative' in name.lower():
            narrative = item
        if 'prompt' in name.lower():
          system_prompt = item

    full_prompt = f"""[INST]

    {system_prompt}

    [EXAMPLE1] => {example1}

    [EXAMPLE2] => {example2}

    [NARRATIVE] => {narrative}

    [/INST]"""
    return full_prompt


def get_fsinference(**kwargs):
    """"""
    for name, item in kwargs.items():
        if "query" in name.lower():
          narrative = item.strip()
        if "task" in name.lower():
          task = item.strip()
        if "prompt" in name.lower():
          sys_prompt = item
        if "threshold" in name.lower():
          t = item
        if isinstance(item,
                      chromadb.api.models.Collection.Collection):
            store = item
        if isinstance(item,
                      transformers.pipelines.text_generation.TextGenerationPipeline):
            pipeline = item
    examples = store.query(query_texts=[narrative],
                          include=["documents"],
                          where={task:1},
                          n_results=2)['documents'][0]
    prompt = get_fsprompt(**dict(narrative=narrative,
                                 example1=examples[0],
                                 example2=examples[1],
                                 sys_prompt=sys_prompt))
    inference = pipeline(prompt,
                         temperature=0.01,
                         do_sample=True,
                         num_return_sequences=1,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=5000)
    pred = format_fsinference(inference=inference, threshold =t)
    return pred

In [ ]:
def get_zsprompt(**kwargs):
    """Zero shot prompt builder"""
    for name, item in kwargs.items():
        if 'narrative' in name.lower():
            narrative = item
        if 'prompt' in name.lower():
            sys_prompt = item
    system_prompt = sys_prompt
    full_prompt = "[INST]\n\n" + system_prompt + "\n\n[NARRATIVE] => " + narrative  + "\n\n[/INST]"
    return full_prompt


def get_zsinference(**kwargs):
    """Run inference given a narrative and return a one-hot list of present anomalies in the narrative.
    """
    for name, item in kwargs.items():
        if "narrative" in name.lower():
            narrative = item.strip()
        if 'prompt' in name.lower():
            sys_prompt = item
        if "threshold" in name.lower():
          t = item
        if isinstance(item,
                      transformers.pipelines.text_generation.TextGenerationPipeline):
            pipeline = item
    prompt = get_zsprompt(**dict(narrative=narrative,
                                 prompt=sys_prompt))
    inference = pipeline(prompt,
                         do_sample=True,
                         temperature=0.01,
                         top_k=10,
                         num_return_sequences=1,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=3000)
    pred = format_zsinference(inference=inference, threshold=t)
    return pred

In [ ]:
def binary_accuracy_per_label(y_true, y_pred):
    correct = y_pred == y_true
    accuracy_per_label = correct.float().mean(axis=0)
    return accuracy_per_label

def binary_accuracy_averaged(y_true, y_pred):
    accuracy_per_label = binary_accuracy_per_label(y_true, y_pred)
    accuracy_averaged = accuracy_per_label.mean()
    return accuracy_averaged

def custom_classification_report(y_true, y_pred):
    report = metrics.classification_report(y_true, y_pred, output_dict=True, target_names=ANOMALY_LABELS, zero_division=0)
    accuracy = binary_accuracy_per_label(y_true, y_pred)
    extended_accuracy_new = np.append(accuracy, [accuracy.mean()] * (len(report) - len(accuracy)))

    updated_report = {}
    for i, class_label in enumerate(report.keys()):
        # Create a new dictionary for the class with binary accuracy
        class_dict = {'binary_accuracy': extended_accuracy_new[i]}

        # Merge this dictionary with the existing metrics for the class
        class_dict.update(report[class_label])

        # Update the main report dictionary
        updated_report[class_label] = class_dict

    return updated_report

In [ ]:
ANOMALY_LABELS = ['Deviation / Discrepancy - Procedural',
                    'Aircraft Equipment',
                    'Conflict',
                    'Inflight Event / Encounter',
                    'ATC Issue',
                    'Deviation - Altitude',
                    'Deviation - Track / Heading',
                    'Ground Event / Encounter',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Ground Incursion',
                    'Airspace Violation',
                    'Deviation - Speed',
                    'Ground Excursion',
                    'No Specific Anomaly Occurred']

In [ ]:
def encode_cell(cell: pd.Series, labels: list) -> pd.Series:
    """Encode the multilabels cell such that the cell content is replaced by \n
    a list of same length as labels and containing 0/1.

    Args:
        cell (pd.Series): cell containing the multilabel target
        labels (list): actual list of labels to classify.

    Returns:
        pd.Series: Expand of the cell with number of cols\n
        equal to number of element in labels.
    """
    cell_anomalies = [item.strip() for item in cell.split(';')]
    splited_cell_anomalies = {label: any(item.startswith(label)
                                         for item in cell_anomalies)
                              for label in labels}
    return pd.Series(splited_cell_anomalies)


def target_encoder(**kwargs):
    """_summary_
    Args:
        kwargs:
            Datasets: kwargs should key-value of the datasets
            target(str): All the passed datasets should contain this column.
            That's the column to encode.
            labels(list): list of the labels to encode.
    """
    data_list = []
    for name, item in kwargs.items():
        if "target" in name.lower():
            target = item
        if "labels" in name.lower():
            labels = tuple(item) if isinstance(item, list) else item
    for name, item in kwargs.items() :
        if isinstance(item, pd.DataFrame):
            data = item.copy()
            encoding_series = data[target].apply(
                lambda cell: encode_cell(cell, labels))
            data[target] = encoding_series.values.tolist()
            data_list.append(data)
    return data_list


def subsample(**kwargs):
  df_list = []
  sub_dfs = []
  for name, item in kwargs.items():
    if 'seed' in name.lower():
      seed = item
    if 'target' in name.lower():
      target = item
    if isinstance(item, int) and 'size' in name.lower():
      size = item
    if isinstance(item, pd.DataFrame):
      data = item.copy()
      data['index'] = range(0,data.shape[0])
      data = data.set_index('index')
      df_list.append(data)

  np.random.seed(seed)
  for frame in df_list:
    indices = []
    frame[target] = frame[target].apply(lambda cell: [1 if b else 0 for b in cell])
    values = np.array(frame[target].values.tolist())
    for i in range(values.shape[1]):
      matching_indices = np.setdiff1d(np.where(values[:,i]==1)[0], indices)
      if matching_indices.shape[0] >= size:
        indices = indices + np.random.choice(matching_indices,
                                             size=size,
                                             replace=False).tolist()
      else:
        indices = indices + np.random.choice(matching_indices,
                                             size=matching_indices.shape[0],
                                             replace=False).tolist()
    sub_dfs.append(frame.iloc[indices])
  return sub_dfs



# LOAD DATA

In [ ]:
%%time
train_data_pkl = pd.read_pickle(os.path.join(WORKING_DIR,"train_data_final.pkl"))[0]  # load pkl data
test_data_pkl = pd.read_pickle(os.path.join(WORKING_DIR,"test_data_final.pkl"))[0]  # load pkl test data
train_data = train_data_pkl[['Narrative', 'Anomaly']].dropna(axis=0, subset=['Narrative','Anomaly'])
test_data = test_data_pkl[['Narrative', 'Anomaly']].dropna(axis=0, subset=['Narrative','Anomaly'])

In [ ]:
raw_labels = train_data.Anomaly.apply(lambda cell: cell.strip().split(';'))
raw_labels = sorted(set(list(chain.from_iterable(raw_labels.values.tolist()))))
extend_labels = []
for prefix in ANOMALY_LABELS:
  for label in raw_labels:
    if label.startswith(prefix):
      extend_labels.append(label)

In [ ]:
train_data_enc, test_data_enc = target_encoder(train=train_data, test=test_data, target='Anomaly', labels=ANOMALY_LABELS)

In [ ]:
test_data.Narrative[1014798]

In [ ]:
sub_train_data, sub_test_data = subsample(seed=7, target='Anomaly', size=100, df1=train_data_enc, df2=test_data_enc)

# CONTENT

## SETUP THE VECTORE STORE

In [ ]:
asrsnlp_client = chromadb.PersistentClient(path=os.path.join(WORKING_DIR,"chromadb"))

In [ ]:
embedder = DefaultEmbeddingFunction()
asrsnlp_collection = asrsnlp_client.get_or_create_collection(name="asrsnlp_collection",
                                                             metadata={"hnsw:space": "cosine"},
                                                             embedding_function=embedder)

In [ ]:
try:
    assert asrsnlp_client.list_collections() != []
except ValueError:
    embedder = DefaultEmbeddingFunction()
    asrsnlp_collection = asrsnlp_client.get_or_create_collection(
    name="asrsnlp_collection",
    metadata={"hnsw:space": "cosine"},
    embedding_function=embedder)
    documents = load_from_df(df=train_data, docs='Narrative',metadata='Anomaly', anomalies=ANOMALY_LABELS)

    asrsnlp_collection.add(
        documents=documents[0][:40000],
        metadatas=documents[1][:40000],
        ids=[f"ID{i}" for i in range(0,40000)]
    )

    asrsnlp_collection.add(
        documents=documents[0][40000:80000],
        metadatas=documents[1][40000:80000],
        ids=[f"ID{i}" for i in range(40000, 80000)]
    )

    asrsnlp_collection.add(
        documents=documents[0][80000:],
        metadatas=documents[1][80000:],
        ids=[f"ID{i}" for i in range(80000,96986)]
    )

## INFERENCE

In [ ]:
THRESHOLD = 0.55

### ZERO SHOT

#### P1

In [ ]:
sys_prompt1 = """As an aviation safety specialist, you will be provided with an input, which is the narrative from Aviation Safety reports of the NASA's ASRS dataset. Tell me if the input corresponds to which anomalies among the following categories:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

Only return the corresponding anomalies and nothing else. Don't mention not corresponding anomalies in your response."""

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=sub_test_data.Narrative[6922],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt1)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

#### P2

In [ ]:
sys_prompt2 = """You are an AI assistant that helps an aviation safety specialist to extract information from aviation safety events reports of the NASA's ASRS dataset.

from the text of a safety event narrative, classify it in one or many of the following categories:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

Only return the corresponding categories and nothing else. Don't mention not corresponding categories in your response."""

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=sub_test_data.Narrative[6922],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt2)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

#### P3

In [ ]:
sys_prompt3 = """You are an AI assistant that helps an aviation safety specialist to extract information from aviation safety events reports of the NASA's ASRS dataset.

from the text of a safety event narrative, classify it in one or many of the following categories:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

Return a python list containing the probability that the input text belong to each category. Each probability is a real number between 0 and 1."""

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=sub_test_data.Narrative[6922],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt3)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

#### P4

In [ ]:
sys_prompt4 = f"""As an aviation security expert, your task is to evaluate the presence or absence of any of the listed issues explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. The list of possible issues includes:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

If the narrative [NARRATIVE] explicitly mentions any case of these issues, you should assign a high probability to that particular issue. Conversely, if the narrative [NARRATIVE] does not explicitly mention any of the listed issues, you should assign a low probability to each one. Your response should be formatted as follows:"""+"""

{'Deviation / Discrepancy – Procedural' : probability,
 'Aircraft Equipment' : probability,
 'Conflict': probability,
 'Inflight Event / Encounter' : probability,
 'ATC Issue' : probability,
 'Deviation – Altitude' : probability,
 'Deviation - Track / Heading' : probability,
 'Ground Event / Encounter' : probability,
 'Flight Deck / Cabin / Aircraft Event' : probability,
 'Ground Incursion' : probability,
 'Airspace Violation' : probability,
 'Deviation – Speed' : probability,
 'Ground Excursion' : probability,
 'No Specific Anomaly Occurred' : probability}."""

In [ ]:
sub_test_data.Anomaly[5110]

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=sub_test_data.Narrative[5110],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt4)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

In [ ]:
test_data.Anomaly[1014798]

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=test_data.Narrative[1014798],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt4)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

#### P5

In [ ]:
sys_prompt5 = f"""As an aviation security expert, your task is to evaluate the presence or absence of any of the listed issues explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. The list of possible issues includes:

0. 'Deviation / Discrepancy - Procedural'
1. 'Aircraft Equipment'
2. 'Conflict'
3. 'Inflight Event / Encounter'
4. 'ATC Issue'
5. 'Deviation - Altitude'
6. 'Deviation - Track / Heading'
7. 'Ground Event / Encounter'
8. 'Flight Deck / Cabin / Aircraft Event'
9. 'Ground Incursion'
10. 'Airspace Violation'
11. 'Deviation - Speed'
12. 'Ground Excursion'
13. 'No Specific Anomaly Occurred'

If the narrative [NARRATIVE] explicitly mentions any case of these issues, you should assign a high probability (above {THRESHOLD}) to that particular issue. Conversely, if the narrative [NARRATIVE] does not explicitly mention any of the listed issues, you should assign a low probability (below {THRESHOLD}) to each one. Your response should be formatted as follows:"""+"""

{'Deviation / Discrepancy – Procedural' : probability,
 'Aircraft Equipment' : probability,
 'Conflict': probability,
 'Inflight Event / Encounter' : probability,
 'ATC Issue' : probability,
 'Deviation – Altitude' : probability,
 'Deviation - Track / Heading' : probability,
 'Ground Event / Encounter' : probability,
 'Flight Deck / Cabin / Aircraft Event' : probability,
 'Ground Incursion' : probability,
 'Airspace Violation' : probability,
 'Deviation – Speed' : probability,
 'Ground Excursion' : probability,
 'No Specific Anomaly Occurred' : probability}."""

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=sub_test_data.Narrative[5110],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt5)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=test_data.Narrative[1014798],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt5)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

In [ ]:
sub_test_data.Anomaly[6922]

In [ ]:
%%time
# SINGLE INFERENCE
result_zs = get_zsinference(narrative=sub_test_data.Narrative[6922],
                            threshold=THRESHOLD,
                            pipeline=pipeline,
                            prompt=sys_prompt5)
print("-------------------\n",
      result_zs[0],
      "\n-------------------\n",
      result_zs[1])

### FEW SHOT

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to evaluate the presence or absence of <<Conflict>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<Conflict>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<Conflict>>. If the narrative [NARRATIVE] explicitly mentions a case of <<Conflict>>, you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<Conflict>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=sub_test_data.Narrative[5110],
                            task='Conflict',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to evaluate the presence or absence of <<Conflict>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<Conflict>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<Conflict>>. If the narrative [NARRATIVE] explicitly mentions a case of <<Conflict>>, you should assign a high probability to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<Conflict>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high. The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=test_data.Narrative[1014798],
                            task='Conflict',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Deviation / Discrepancy - Procedural>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Deviation / Discrepancy - Procedural>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Deviation / Discrepancy – Procedural>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Deviation / Discrepancy – Procedural>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=test_data.Narrative[1014798],
                            task='Deviation / Discrepancy - Procedural',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Deviation / Discrepancy - Procedural>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Deviation / Discrepancy - Procedural>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Deviation / Discrepancy – Procedural>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Deviation / Discrepancy – Procedural>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=sub_test_data.Narrative[5110],
                            task='Deviation / Discrepancy - Procedural',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Aircraft Equipment>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Aircraft Equipment>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Aircraft Equipment>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Aircraft Equipment>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=test_data.Narrative[1014798],
                            task='Aircraft Equipment',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Aircraft Equipment>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Aircraft Equipment>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Aircraft Equipment>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Aircraft Equipment>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=sub_test_data.Narrative[5110],
                            task='Aircraft Equipment',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Airspace Violation>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Airspace Violation>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Airspace Violation>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Airspace Violation>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=test_data.Narrative[1014798],
                            task='Airspace Violation',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Airspace Violation>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Airspace Violation>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Airspace Violation>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Airspace Violation>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=sub_test_data.Narrative[5110],
                            task='Airspace Violation',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Ground Excursion>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Ground Excursion>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Ground Excursion>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Ground Excursion>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=test_data.Narrative[1014798],
                            task='Ground Excursion',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt = f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Ground Excursion>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Ground Excursion>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Ground Excursion>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD})  to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Ground Excursion>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=sub_test_data.Narrative[5110],
                            task='Ground Excursion',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt =f"""As an aviation security expert, your task is to evaluate whether there is <<No Specific Anomaly Occurred>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives where there is <<No Specific Anomaly Occurred>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether there is <<No Specific Anomaly Occurred>>. If there is <<No Specific Anomaly Occurred>> in the narrative [NARRATIVE] you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does explicitly mention a specific anomaly, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=test_data.Narrative[1014798],
                            task='No Specific Anomaly Occurred',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

In [ ]:
%%time
# SINGLE INFERENCE
sys_prompt =f"""As an aviation security expert, your task is to evaluate whether there is <<No Specific Anomaly Occurred>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives where there is <<No Specific Anomaly Occurred>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether there is <<No Specific Anomaly Occurred>>. If there is <<No Specific Anomaly Occurred>> in the narrative [NARRATIVE] you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does explicitly mention a specific anomaly, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows: """+"""{'yes': 0.85, 'no': 0.15}."""
result_fs = get_fsinference(query=sub_test_data.Narrative[5110],
                            task='No Specific Anomaly Occurred',
                            threshold=THRESHOLD,
                            store=asrsnlp_collection,
                            pipeline=pipeline,
                            prompt=sys_prompt)
print("-------------------\n",
      result_fs[0],
      "\n-------------------\n",
      result_fs[1])

# EVALUATION

In [ ]:
ytrue = np.array(sub_test_data.Anomaly.values.tolist())

## ZERO SHOT

### P5

In [ ]:
preds_zs_p5 = sub_test_data.Narrative.apply(lambda cell: get_zsinference(narrative=cell,
                                                                     threshold=THRESHOLD,
                                                                      pipeline=pipeline,
                                                                      prompt=sys_prompt5)[0])
ypredp5 = preds_zs_p5.values.tolist()
joblib.dump(ypredp5, os.path.join(WORKING_DIR,'ypred_p5_zs.joblib'))
print(metrics.classification_report(ytrue, ypredp5, target_names=ANOMALY_LABELS))
format_errors = (np.count_nonzero(np.where(np.sum(ypredp5, axis=1) == 0)[0]) / ypredp5.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

## FEW SHOT

In [ ]:
tasks = [task.strip() for task in ANOMALY_LABELS]
prompts = [f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Deviation / Discrepancy - Procedural>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Deviation / Discrepancy - Procedural>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Deviation / Discrepancy – Procedural>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Deviation / Discrepancy – Procedural>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Aircraft Equipment>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Aircraft Equipment>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Aircraft Equipment>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Aircraft Equipment>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to evaluate the presence or absence of <<Conflict>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<Conflict>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<Conflict>>. If the narrative [NARRATIVE] explicitly mentions a case of <<Conflict>>, you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<Conflict>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to evaluate the presence or absence of <<Inflight Event / Encounter>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<Inflight Event / Encounter>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<Inflight Event / Encounter>>. If the narrative [NARRATIVE] explicitly mentions a case of <<Inflight Event / Encounter>>, you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<Inflight Event / Encounter>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to evaluate the presence or absence of <<ATC Issue>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<ATC Issue>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<ATC Issue>>. If the narrative [NARRATIVE] explicitly mentions a case of <<ATC Issue>>, you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<ATC Issue>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Deviation - Altitude>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Deviation - Altitude>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Deviation - Altitude>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Deviation – Altitude>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Deviation - Track / Heading>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Deviation - Track / Heading>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Deviation - Track / Heading>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Deviation - Track / Heading>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to evaluate the presence or absence of <<Ground Event / Encounter>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<Ground Event / Encounter>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<Ground Event / Encounter>>. If the narrative [NARRATIVE] explicitly mentions a case of <<Ground Event / Encounter>>, you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<Ground Event / Encounter>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to evaluate the presence or absence of <<Flight Deck / Cabin / Aircraft Event>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives that explicitly mention a case of <<Flight Deck / Cabin / Aircraft Event>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether it explicitly mentions a case of <<Flight Deck / Cabin / Aircraft Event>>. If the narrative [NARRATIVE] explicitly mentions a case of <<Flight Deck / Cabin / Aircraft Event>>, you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not explicitly mention a case of <<Flight Deck / Cabin / Aircraft Event>>, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Ground Incursion>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Ground Incursion>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Ground Incursion>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Ground Incursion>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Airspace Violation>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Airspace Violation>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Airspace Violation>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Airspace Violation>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Deviation - Speed>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Deviation - Speed>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Deviation - Speed>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Deviation - Speed>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to determine whether the narrative [NARRATIVE] explicitly mentions a <<Ground Excursion>> issue and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] demonstrate narratives that explicitly discuss such <<Ground Excursion>> issues. Use these examples as a guide to analyze the narrative [NARRATIVE] and determine if it explicitly addresses a <<Ground Excursion>> issue. If the narrative [NARRATIVE] does mention such an issue explicitly, assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does not mention a <<Ground Excursion>> issue explicitly, assign a low probability (below {THRESHOLD}) to the "yes" response and a high probability (above {THRESHOLD}) to the "no" response. Ensure that the sum of these two probabilities equals 1. Format your response as follows:"""+""" {'yes': 0.85, 'no': 0.15}.""",
           f"""As an aviation security expert, your task is to evaluate whether there is <<No Specific Anomaly Occurred>> explicitly mentioned in the narrative [NARRATIVE] and assign probabilities accordingly. Examples [EXAMPLE1] and [EXAMPLE2] illustrate narratives where there is <<No Specific Anomaly Occurred>>. You should draw inspiration from these examples to analyze the narrative [NARRATIVE] and determine whether there is <<No Specific Anomaly Occurred>>. If there is <<No Specific Anomaly Occurred>> in the narrative [NARRATIVE] you should assign a high probability (above {THRESHOLD}) to the "yes" response and a low probability (below {THRESHOLD}) to the "no" response. Conversely, if the narrative [NARRATIVE] does explicitly mention a specific anomaly, the probability of a "yes" response should be low (below {THRESHOLD}) and the probability of a "no" response should be high (above {THRESHOLD}). The sum of the two probabilities should be equal to 1. Your response should be formatted as follows:"""+""" {'yes': 0.85, 'no': 0.15}."""]

### Deviation / Discrepancy - Procedural

In [ ]:
preds_fs_0 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[0],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[0])[0])
ypredfs0 = preds_fs_0.values.tolist()
joblib.dump(ypredfs0, os.path.join(WORKING_DIR,'ypred_fs0.joblib'))
print(metrics.classification_report(ytrue[:,0], ypredfs0[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs0, axis=1) == 0)[0]) / ypredfs0.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Aircraft Equipment

In [ ]:
preds_fs_1 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[1],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[1])[0])
ypredfs1 = preds_fs_1.values.tolist()
joblib.dump(ypredfs1, os.path.join(WORKING_DIR,'ypred_fs1.joblib'))
print(metrics.classification_report(ytrue[:,1], ypredfs1[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs1, axis=1) == 0)[0]) / ypredfs1.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Conflict

In [ ]:
preds_fs_2 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[2],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[2])[0])
ypredfs2 = preds_fs_2.values.tolist()
joblib.dump(ypredfs2, os.path.join(WORKING_DIR,'ypred_fs2.joblib'))
print(metrics.classification_report(ytrue[:,2], ypredfs2[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs2, axis=1) == 0)[0]) / ypredfs2.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Inflight Event / Encounter

In [ ]:
preds_fs_3 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[3],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[3])[0])
ypredfs3 = preds_fs_3.values.tolist()
joblib.dump(ypredfs3, os.path.join(WORKING_DIR,'ypred_fs3.joblib'))
print(metrics.classification_report(ytrue[:,3], ypredfs3[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs3, axis=1) == 0)[0]) / ypredfs3.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### ATC Issue

In [ ]:
preds_fs_4 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[4],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[4])[0])
ypredfs4 = preds_fs_4.values.tolist()
joblib.dump(ypredfs4, os.path.join(WORKING_DIR,'ypred_fs4.joblib'))
print(metrics.classification_report(ytrue[:,4], ypredfs4[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs4, axis=1) == 0)[0]) / ypredfs4.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Deviation - Altitude


In [ ]:
preds_fs_5 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[5],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[5])[0])
ypredfs5 = preds_fs_5.values.tolist()
joblib.dump(ypredfs5, os.path.join(WORKING_DIR,'ypred_fs5.joblib'))
print(metrics.classification_report(ytrue[:,5], ypredfs5[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs5, axis=1) == 0)[0]) / ypredfs5.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Deviation - Track / Heading

In [ ]:
preds_fs_6 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[6],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[6])[0])
ypredfs6 = preds_fs_6.values.tolist()
joblib.dump(ypredfs6, os.path.join(WORKING_DIR,'ypred_fs6.joblib'))
print(metrics.classification_report(ytrue[:,6], ypredfs6[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs6, axis=1) == 0)[0]) / ypredfs6.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Ground Event / Encounter'

In [ ]:
preds_fs_7 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[7],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[7])[0])
ypredfs7 = preds_fs_7.values.tolist()
joblib.dump(ypredfs7, os.path.join(WORKING_DIR,'ypred_fs7.joblib'))
print(metrics.classification_report(ytrue[:,7], ypredfs7[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs7, axis=1) == 0)[0]) / ypredfs7.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Flight Deck / Cabin / Aircraft Event

In [ ]:
preds_fs_8 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[8],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[8])[0])
ypredfs8 = preds_fs_8.values.tolist()
joblib.dump(ypredfs8, os.path.join(WORKING_DIR,'ypred_fs8.joblib'))
print(metrics.classification_report(ytrue[:,8], ypredfs8[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs8, axis=1) == 0)[0]) / ypredfs8.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Ground Incursion

In [ ]:
preds_fs_9 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[9],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[9])[0])
ypredfs9 = preds_fs_9.values.tolist()
joblib.dump(ypredfs9, os.path.join(WORKING_DIR,'ypred_fs9.joblib'))
print(metrics.classification_report(ytrue[:,9], ypredfs9[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs9, axis=1) == 0)[0]) / ypredfs9.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Airspace Violation

In [ ]:
preds_fs_10 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[10],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[10])[0])
ypredfs10 = preds_fs_10.values.tolist()
joblib.dump(ypredfs10, os.path.join(WORKING_DIR,'ypred_fs10.joblib'))
print(metrics.classification_report(ytrue[:,10], ypredfs10[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs10, axis=1) == 0)[0]) / ypredfs10.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

###  Deviation - Speed

In [ ]:
preds_fs_11 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[11],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[11])[0])
ypredfs11 = preds_fs_11.values.tolist()
joblib.dump(ypredfs11, os.path.join(WORKING_DIR,'ypred_fs11.joblib'))
print(metrics.classification_report(ytrue[:,11], ypredfs11[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs11, axis=1) == 0)[0]) / ypredfs11.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### Ground Excursion

In [ ]:
preds_fs_12 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[12],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[12])[0])
ypredfs12 = preds_fs_12.values.tolist()
joblib.dump(ypredfs12, os.path.join(WORKING_DIR,'ypred_fs12.joblib'))
print(metrics.classification_report(ytrue[:,12], ypredfs12[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs12, axis=1) == 0)[0]) / ypredfs12.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)

### No Specific Anomaly Occurred

In [ ]:
preds_fs_13 = sub_test_data.Narrative.apply(lambda cell: get_fsinference(query=cell,
                                                                      task=tasks[13],
                                                                      threshold=THRESHOLD,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline,
                                                                      prompt=prompts[13])[0])
ypredfs13 = preds_fs_13.values.tolist()
joblib.dump(ypredfs13, os.path.join(WORKING_DIR,'ypred_fs13.joblib'))
print(metrics.classification_report(ytrue[:,13], ypredfs13[:,0]))
format_errors = (np.count_nonzero(np.where(np.sum(ypredfs13, axis=1) == 0)[0]) / ypredfs13.shape[0]) * 100
print('--------------------------------------------\n',"%format error : ", format_errors)